In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.sparse.linalg import svds

In [2]:
# Chargement du dataset
df=pd.read_csv('/Users/CAMARA110459/Desktop/DOC_JOUR/movie_df.csv',sep=',')
df.head(10)

,movieId,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,tag1,tag2,directorAvgRating,actor1AvgRating,actor2AvgRating,writerAvgRating,tag0AvgRating,tag1AvgRating,tag2AvgRating,genresAvgRating
0,1,tt0114709,Toy Story,Toy Story,1995,81,"adventure,animation,comedy",nm0005124,"nm0005124,nm0230032,nm0004056,nm0710020,nm0923...",4.15,...,244.0,786.0,3.690000,3.513235,3.275000,3.635714,3.750000,3.457143,4.100000,3.193560
1,2,tt0113497,Jumanji,Jumanji,1995,104,"adventure,comedy,family",nm0002653,"nm0378144,nm0852430,nm0833164,nm0885575",3.55,...,584.0,204.0,3.277778,3.153125,3.277778,3.480000,3.008333,2.861111,3.152174,3.113986
2,3,tt0113228,Grumpier Old Men,Grumpier Old Men,1995,101,"comedy,romance",nm0222043,nm0425756,3.35,...,901.0,902.0,3.115000,3.336842,3.377273,2.875000,2.911765,2.745000,2.761111,3.172996
3,4,tt0114885,Waiting to Exhale,Waiting to Exhale,1995,124,"comedy,drama,romance",nm0001845,"nm0573334,nm0060103",3.00,...,201.0,439.0,2.850000,3.000000,2.983333,2.925000,3.138333,3.105714,3.096552,3.216009
4,5,tt0113041,Father of the Bride Part II,Father of the Bride Part II,1995,106,"comedy,family,romance",nm0796124,"nm0352443,nm0329304,nm0583600,nm0796124",3.05,...,383.0,902.0,3.021429,3.165000,3.403125,3.485714,2.911765,3.162500,2.761111,3.151227
5,6,tt0113277,Heat,Heat,1995,170,"action,crime,drama",nm0000520,nm0000520,4.15,...,498.0,481.0,3.486364,3.498276,3.338235,3.633333,3.462500,3.415385,3.362500,3.184678
6,7,tt0114319,Sabrina,Sabrina,1995,127,"comedy,drama,romance",nm0001628,"nm0853138,nm0000697,nm0499626,nm0070660,nm0713128",3.15,...,864.0,865.0,3.381579,3.374000,2.925000,3.383333,2.798077,3.201111,3.158696,3.216009
7,8,tt0112302,Tom and Huck,Tom and Huck,1995,97,"adventure,comedy,drama",nm0382072,"nm0878494,nm0814085,nm0521739",2.75,...,107.0,1062.0,2.712500,2.900000,3.033333,3.150000,3.008333,3.223684,3.253125,3.178767
8,9,tt0114576,Sudden Death,Sudden Death,1995,111,"action,crime,thriller",nm0001382,"nm0049945,nm0704164",2.90,...,447.0,609.0,2.982353,2.760000,3.100000,2.900000,3.015493,2.967391,3.375000,3.098343
9,10,tt0113189,GoldenEye,GoldenEye,1995,130,"action,adventure,thriller",nm0132709,"nm0001220,nm0289833,nm0128997,nm0270761",3.60,...,2.0,151.0,3.172727,3.200000,3.138889,3.366667,3.083333,3.600000,3.346875,3.070256


#### La Popularité selon l'année 

In [4]:
def recommend_movies(year=None, num_recommendations=5, min_votes=1000):
    """
    Recommande les films les mieux notés en filtrant par :
    - Année de sortie (optionnelle)
    - Nombre minimum de votes pour garantir une certaine popularité

    Paramètres :
    - year (int, optionnel) : Année de sortie à filtrer.
    - num_recommendations (int) : Nombre de films à recommander.
    - min_votes (int) : Nombre minimal de votes requis pour qu’un film soit pris en compte.

    Retourne :
    - DataFrame des films recommandés avec titre, année et note moyenne.
    """

    filtered_df = df.copy()

    # Filtrer par année si spécifiée
    if year is not None:
        filtered_df = filtered_df[filtered_df['startYear'] == year]

    # Filtrer par nombre minimum de votes
    filtered_df = filtered_df[filtered_df['numVotes'] >= min_votes]

    # Trier par note moyenne décroissante et sélectionner les meilleures entrées
    recommended = (
        filtered_df.sort_values(by='averageRating', ascending=False)
        .head(num_recommendations)
        [['primaryTitle', 'startYear', 'averageRating', 'numVotes']]
    )

    return recommended

# Exemple : recommander 10 films de 2010 avec au moins 1000 votes
recommend_movies(2010, 6, min_votes=1000)

,primaryTitle,startYear,averageRating,numVotes
14063,Inception,2010,4.40,2619603
15384,Incendies,2010,4.15,214949
13961,Toy Story 3,2010,4.15,913594
13522,Shutter Island,2010,4.10,1507754
17885,Udaan,2010,4.05,48003
13657,How to Train Your Dragon,2010,4.05,823103


### Récommation des film les mieux notés pour un genre et une année données

In [5]:
def recommander_top10_par_genre_et_annee(genre, annee):
    # Filtrer les films du genre et de l'année spécifiée
    selection = df[
        df['genres'].str.contains(genre, case=False, na=False) &
        (df['startYear'] == annee)
    ]

    # Vérifier s’il y a des résultats
    if selection.empty:
        print(f"Aucun film trouvé pour le genre '{genre}' en {annee}.")
        return

    # Trier par note moyenne et afficher le top 10
    top10 = selection.sort_values('averageRating', ascending=False).head(10)

    print(f"\nTop 10 des films du genre '{genre}' sortis en {annee} :\n")
    print(top10[['primaryTitle', 'genres', 'startYear', 'averageRating','numVotes']])

# Exemple d'appel :
recommander_top10_par_genre_et_annee('Action', 2008)



Top 10 des films du genre 'Action' sortis en 2008 :

                                   primaryTitle                      genres  \
12280                                    Ip Man      action,biography,drama   
13146                             Love Exposure         action,comedy,drama   
22552     The Garden of Sinners: Paradox Spiral      action,animation,crime   
11707                                  Iron Man     action,adventure,sci-fi   
13431                                The Chaser          action,crime,drama   
11714                                     Taken       action,crime,thriller   
19186  Gurren Lagann the Movie: Childhood's End     action,animation,comedy   
11754                             Kung Fu Panda  action,adventure,animation   
11559                              Jodhaa Akbar        action,drama,history   
12291                  Mesrine: Killer Instinct      action,biography,crime   

       startYear  averageRating  numVotes  
12280       2008           4.00 